# MVP Agents: Sistema de Trading com Agentes Cooperativos

Este notebook implementa um sistema completo de trading com dois agentes cooperativos:
- **TraderAgent**: Gera propostas de ordens baseadas em estratégias
- **RiskAgent**: Valida e filtra propostas com base em limites de risco

## Objetivos

1. Simular dados sintéticos (spot, futuros, opções)
2. Implementar Black-Scholes e greeks
3. Implementar estratégias de trading
4. Executar backtest walk-forward
5. Gerar métricas e visualizações

## 1. Configuração e Imports

In [ ]:
import sys
from pathlib import Path
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from tqdm import tqdm

# Adicionar src ao path
sys.path.insert(0, str(Path.cwd() / 'src'))

# Imports dos módulos
from data_loader import DataLoader
from pricing import BlackScholes, numerical_greeks_check
from agents import TraderAgent, RiskAgent, PortfolioManager
from execution import ExecutionSimulator
from backtest import BacktestEngine
from utils import StructuredLogger, calculate_metrics, get_version_info

# Configuração de visualização
plt.style.use('seaborn-v0_8-darkgrid')
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

print("✓ Imports concluídos")

In [ ]:
# Carregar configuração
with open('config.json', 'r') as f:
    config = json.load(f)

print("Configuração carregada:")
print(json.dumps(config, indent=2))

In [ ]:
# Informações de versão
version_info = get_version_info()
print("Informações de Versão:")
for key, value in version_info.items():
    print(f"  {key}: {value}")

## 2. Geração/Carregamento de Dados

In [ ]:
# Inicializar DataLoader
data_loader = DataLoader(data_dir='data')

# Gerar dados sintéticos
print("Gerando dados sintéticos...")

spot_df = data_loader.generate_synthetic_spot(
    tickers=['AAPL', 'MSFT'],
    start_date='2024-01-01',
    end_date='2024-12-31',
    seed=42
)

futures_df = data_loader.generate_synthetic_futures(
    contracts=['ESZ25'],
    start_date='2024-01-01',
    end_date='2024-12-31',
    seed=42
)

options_df = data_loader.generate_synthetic_options_chain(
    underlying='AAPL',
    start_date='2024-01-01',
    end_date='2024-12-31',
    seed=42
)

print(f"✓ Spot: {len(spot_df)} registros")
print(f"✓ Futuros: {len(futures_df)} registros")
print(f"✓ Opções: {len(options_df)} registros")

# Salvar CSVs de exemplo
data_loader.save_csv(spot_df, 'spot.csv')
data_loader.save_csv(futures_df, 'futures.csv')
data_loader.save_csv(options_df, 'options_chain.csv')

print("\nDados salvos em data/")
print(spot_df.head())

### Como substituir por dados reais:

```python
# Carregar dados reais de CSV
spot_df = data_loader.load_spot_csv('data/spot_real.csv')
futures_df = data_loader.load_futures_csv('data/futures_real.csv')
options_df = data_loader.load_options_csv('data/options_real.csv')
```

## 3. Testes de Precificação

In [ ]:
# Teste básico de Black-Scholes
S = 150.0
K = 150.0
T = 0.25  # 3 meses
r = 0.05
sigma = 0.25

call_price = BlackScholes.price(S, K, T, r, sigma, 'C')
put_price = BlackScholes.price(S, K, T, r, sigma, 'P')

print(f"Preço Call (ATM, 3 meses, IV=25%): R$ {call_price:.2f}")
print(f"Preço Put (ATM, 3 meses, IV=25%): R$ {put_price:.2f}")

# Greeks
greeks = BlackScholes.all_greeks(S, K, T, r, sigma, 'C')
print("\nGreeks (Call):")
for greek, value in greeks.items():
    print(f"  {greek}: {value:.6f}")

## 4. Setup dos Agentes

In [ ]:
# Criar logger
logger = StructuredLogger(log_dir='logs')

# Setup do backtest engine
backtest_engine = BacktestEngine(config, logger)
backtest_engine.setup(data_loader, BlackScholes)
backtest_engine.load_data(spot_df, futures_df, options_df)

print("✓ Backtest engine configurado")

## 5. Execução do Backtest

In [ ]:
# Executar backtest
print("Executando backtest...")

# Resetar componentes
backtest_engine.portfolio_manager = PortfolioManager(config['nav'])
backtest_engine.risk_agent = RiskAgent(backtest_engine.portfolio_manager, config, logger)
backtest_engine.execution_simulator = ExecutionSimulator(config, logger)

# Executar
backtest_engine.run_simple()

print("\n✓ Backtest concluído")

## 6. Análise de Resultados

In [ ]:
# Métricas agregadas
metrics = backtest_engine.get_aggregate_metrics()

print("Métricas de Performance:")
print("=" * 50)
for metric, value in metrics.items():
    if 'ratio' in metric.lower() or 'rate' in metric.lower():
        print(f"{metric:20s}: {value:>10.4f}")
    elif 'return' in metric.lower() or 'drawdown' in metric.lower() or 'volatility' in metric.lower():
        print(f"{metric:20s}: {value:>10.2f}%")
    else:
        print(f"{metric:20s}: {value:>10.0f}")

# Salvar métricas
Path('output').mkdir(exist_ok=True)
metrics_df = pd.DataFrame([metrics])
metrics_df.to_csv('output/metrics.csv', index=False)
print("\n✓ Métricas salvas em output/metrics.csv")

In [ ]:
# Snapshots do portfólio
portfolio_snapshots = backtest_engine.get_portfolio_snapshots_df()

if not portfolio_snapshots.empty:
    portfolio_snapshots['timestamp'] = pd.to_datetime(portfolio_snapshots['timestamp'])
    portfolio_snapshots = portfolio_snapshots.sort_values('timestamp')
    
    print("Evolução do NAV:")
    print(f"  NAV inicial: R$ {portfolio_snapshots['nav'].iloc[0]:,.2f}")
    print(f"  NAV final: R$ {portfolio_snapshots['nav'].iloc[-1]:,.2f}")
    
    # Salvar
    portfolio_snapshots.to_csv('output/portfolio_snapshots.csv', index=False)
    print("\n✓ Snapshots salvos")
else:
    print("Nenhum snapshot gerado")

In [ ]:
# Ordens e fills
orders_df = backtest_engine.execution_simulator.get_orders_dataframe()
fills_df = backtest_engine.execution_simulator.get_fills_dataframe()

if not orders_df.empty:
    orders_df['timestamp'] = pd.to_datetime(orders_df['timestamp'])
    orders_df.to_csv('output/orders.csv', index=False)
    print(f"✓ {len(orders_df)} ordens salvas")
else:
    print("Nenhuma ordem gerada")

if not fills_df.empty:
    fills_df['timestamp'] = pd.to_datetime(fills_df['timestamp'])
    fills_df.to_csv('output/fills.csv', index=False)
    print(f"✓ {len(fills_df)} fills salvos")
else:
    print("Nenhum fill gerado")

## 7. Visualizações

In [ ]:
# Gráfico: Evolução do NAV
if not portfolio_snapshots.empty:
    fig, ax = plt.subplots(figsize=(12, 6))
    
    portfolio_snapshots_sorted = portfolio_snapshots.sort_values('timestamp')
    ax.plot(portfolio_snapshots_sorted['timestamp'], portfolio_snapshots_sorted['nav'], 
            linewidth=2, label='NAV')
    
    ax.axhline(y=config['nav'], color='r', linestyle='--', alpha=0.5, label='NAV Inicial')
    ax.set_xlabel('Data')
    ax.set_ylabel('NAV (R$)')
    ax.set_title('Evolução do Patrimônio Líquido')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('output/nav_evolution.png', dpi=150)
    plt.show()
    
    print("✓ Gráfico salvo")

In [ ]:
print("=" * 70)
print("RESUMO DO BACKTEST")
print("=" * 70)

if metrics:
    print(f"\n📊 MÉTRICAS PRINCIPAIS:")
    print(f"  Retorno Total: {metrics.get('total_return', 0):.2f}%")
    print(f"  Sharpe Ratio: {metrics.get('sharpe_ratio', 0):.4f}")
    print(f"  Max Drawdown: {metrics.get('max_drawdown', 0):.2f}%")
    print(f"  Total de Trades: {metrics.get('total_trades', 0)}")

print(f"\n📁 ARQUIVOS GERADOS:")
print(f"  output/orders.csv")
print(f"  output/fills.csv")
print(f"  output/portfolio_snapshots.csv")
print(f"  output/metrics.csv")

print("\n" + "=" * 70)